In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import statsmodels.stats.multitest
import operator
import cptac.utils as u


/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
cptac warning: Your version of cptac (0.8.2) is out-of-date. Latest is 0.8.3. Please run 'pip install --upgrade cptac' to update it. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 7)


In [37]:
sup18 = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable18.csv")
sup18["phospho_site"] =  sup18["RefSeq"] + "_" + sup18["Phosphosite"]
sup18 = sup18[["Gene Name","FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)", "phospho_site","RefSeq"]]
sup18 = sup18.rename(columns = {"FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)" : "Significant", "Gene Name" : "Gene_Name"})

#sup18 = sup18.replace(r' ', "_", regex = True)
sup18

,Gene_Name,Significant,phospho_site,RefSeq
0,FAM184B,True,NP_056503_S976s,NP_056503
1,TRIM28,True,NP_005753_S489s,NP_005753
2,ARHGEF35,True,NP_001003702_S450s,NP_001003702
3,RPS6KA5,True,NP_004746_S212s,NP_004746
4,EIF2AK4,True,NP_001013725_S572s,NP_001013725
...,...,...,...,...
25856,FAM171A1,False,NP_001010924_S525s,NP_001010924
25857,TRIM2,False,NP_001123539_S424s S428s,NP_001123539
25858,MAP2,False,NP_002365_T1649t S1653s,NP_002365
25859,MAP2,False,NP_002365_S1155s,NP_002365


In [38]:
sup18[sup18["Gene_Name"] == "FOXO3"]
#AKT2_S34, GSK3B_S9, not sig in original paper.
#FOXO3 not in data 

,Gene_Name,Significant,phospho_site,RefSeq


In [56]:

sup18_T = sup18.loc[sup18["Significant"] == True]
sup18_T = sup18_T.set_index("phospho_site")
sup18_T.head(32)

,Gene_Name,Significant,RefSeq
phospho_site,,,
NP_056503_S976s,FAM184B,True,NP_056503
NP_005753_S489s,TRIM28,True,NP_005753
NP_001003702_S450s,ARHGEF35,True,NP_001003702
NP_004746_S212s,RPS6KA5,True,NP_004746
NP_001013725_S572s,EIF2AK4,True,NP_001013725
NP_056992_S136s,SRRT,True,NP_056992
XP_003960280_S441s,BCL9,True,XP_003960280
NP_001135452_S899s,TP53BP1,True,NP_001135452
NP_001005751_S836s,FAM21A,True,NP_001005751


# Get phosphoproteomic data from paper

In [40]:
df = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable04.csv")
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,27]
df = df.drop(df.columns[cols],axis=1)
df["phospho_site"] =  df["accession_number"] + "_" + df["variableSites"]
df2 = df.set_index("phospho_site")
#df1 = df[df['phospho_site'].isin(cols_62)]
df2

,variableSites,accession_number,geneName,AO-A12D.01TCGA,C8-A131.01TCGA,AO-A12B.01TCGA,E2-A10A.02TCGA,BH-A18Q.02TCGA,C8-A130.02TCGA,C8-A138.03TCGA,...,AO-A12B.34TCGA,A2-A0SW.35TCGA,AO-A0JL.35TCGA,BH-A0BV.35TCGA,A2-A0YM.36TCGA,BH-A0C7.36TCGA,A2-A0SX.36TCGA,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
phospho_site,,,,,,,,,,,,,,,,,,,,,
NP_001611_S1007s,S1007s,NP_001611,AHNAK,-0.919089,-2.565861,0.177221,0.203602,-0.086607,-2.700983,1.301494,...,0.805357,4.389396,1.338505,3.450978,0.200324,1.728556,1.092898,-1.153228,0.234135,2.595100
NP_001611_S1010s,S1010s,NP_001611,AHNAK,-1.271514,-1.726458,-0.320742,-0.523359,-0.315189,-2.208631,1.820337,...,-0.124880,0.070029,0.495032,0.776968,1.017243,2.213385,0.444430,-0.331621,0.347756,2.527968
NP_001611_S1023s,S1023s,NP_001611,AHNAK,NaN,NaN,NaN,NaN,NaN,NaN,-0.302638,...,0.880172,0.812259,0.364182,1.626879,1.985442,-0.342381,1.767384,NaN,NaN,NaN
NP_001611_S1042s,S1042s,NP_001611,AHNAK,-2.298519,-0.370305,-1.169382,0.330244,0.819633,-0.419166,1.570435,...,-0.795385,0.797827,1.628386,1.382329,NaN,NaN,NaN,NaN,NaN,NaN
NP_001611_S1068s,S1068s,NP_001611,AHNAK,-0.802335,-2.023904,-1.223738,-0.465794,-1.585542,-1.411369,1.394314,...,-0.915370,2.065803,3.490470,2.797509,0.485597,2.001793,-0.352144,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_543009_T210t,T210t,NP_543009,GPR78,0.014945,2.483162,-0.324249,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NP_001017372_S217s T218t T224t,S217s T218t T224t,NP_001017372,SLC27A6,-0.698553,-1.693688,-0.175210,0.729908,0.613301,-0.054275,NaN,...,0.504689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NP_954857_T47t,T47t,NP_954857,C19ORF48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df62 = pd.merge(sup18_T, df2, on = "phospho_site", how = "inner")
df62.columns = df62.columns.str.replace(r'...TCGA', "", regex = True)
df62["Gene_Psite"] =  df62["Gene_Name"] + "_" + df62["variableSites"]
df62 = df62.set_index("Gene_Psite")
df62

,Gene_Name,Significant,RefSeq,variableSites,accession_number,geneName,AO-A12D,C8-A131,AO-A12B,E2-A10A,...,AO-A12B,A2-A0SW,AO-A0JL,BH-A0BV,A2-A0YM,BH-A0C7,A2-A0SX,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
Gene_Psite,,,,,,,,,,,,,,,,,,,,,
FAM184B_S976s,FAM184B,True,NP_056503,S976s,NP_056503,FAM184B,-4.916841,-2.966657,-2.954333,-1.798008,...,-2.441299,NaN,NaN,NaN,-3.006837,-2.146746,-4.983482,3.717143,-1.413374,-2.761566
TRIM28_S489s,TRIM28,True,NP_005753,S489s,NP_005753,TRIM28,-2.624999,0.519513,-1.597210,1.724957,...,-1.151106,-0.828894,-2.297088,0.855144,-0.939327,0.467334,-0.956579,3.014077,2.552447,1.402929
ARHGEF35_S450s,ARHGEF35,True,NP_001003702,S450s,NP_001003702,ARHGEF35,-0.017487,0.365749,-0.562710,0.282548,...,-0.338030,-0.228925,-1.910580,-2.392154,NaN,NaN,NaN,NaN,NaN,NaN
RPS6KA5_S212s,RPS6KA5,True,NP_004746,S212s,NP_004746,RPS6KA5,0.157644,0.965682,0.026429,-0.107248,...,0.760187,-2.253563,-2.401767,-1.397919,0.168627,-0.127457,-1.482959,NaN,NaN,NaN
EIF2AK4_S572s,EIF2AK4,True,NP_001013725,S572s,NP_001013725,EIF2AK4,-0.162349,-0.836640,-1.160616,NaN,...,-0.801032,NaN,NaN,NaN,-0.631002,-0.632279,-1.815198,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTS1_S13s,INTS1,True,NP_001073922,S13s,NP_001073922,INTS1,NaN,NaN,NaN,-2.360498,...,NaN,-1.142280,-1.226138,-0.960937,NaN,NaN,NaN,NaN,NaN,NaN
RBM10_S596s,RBM10,True,NP_001191397,S596s,NP_001191397,NaN,NaN,NaN,NaN,1.057205,...,-1.352963,NaN,NaN,NaN,-0.547437,0.345710,-1.152721,-1.333082,-1.232017,-1.546246
KCTD1_S617s S620s,KCTD1,True,NP_001136202,S617s S620s,NP_001136202,KCTD1,-0.780714,0.385914,0.263137,-0.342441,...,0.386115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
cols = ["Significant","RefSeq","accession_number","geneName","Gene_Name","variableSites"]
df62 = df62.drop(cols, axis = 1) 
df62.head()

,AO-A12D,C8-A131,AO-A12B,E2-A10A,BH-A18Q,C8-A130,C8-A138,E2-A154,BH-A0BZ,A8-A09I,...,AO-A12B,A2-A0SW,AO-A0JL,BH-A0BV,A2-A0YM,BH-A0C7,A2-A0SX,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
Gene_Psite,,,,,,,,,,,,,,,,,,,,,
FAM184B_S976s,-4.916841,-2.966657,-2.954333,-1.798008,-3.448592,2.449975,-4.629509,0.906083,-3.190797,-3.790538,...,-2.441299,NaN,NaN,NaN,-3.006837,-2.146746,-4.983482,3.717143,-1.413374,-2.761566
TRIM28_S489s,-2.624999,0.519513,-1.597210,1.724957,-0.394081,-2.106161,1.506175,2.703867,-0.561201,-1.867217,...,-1.151106,-0.828894,-2.297088,0.855144,-0.939327,0.467334,-0.956579,3.014077,2.552447,1.402929
ARHGEF35_S450s,-0.017487,0.365749,-0.562710,0.282548,-1.379211,-1.236422,-1.994830,0.811225,-2.304321,-2.450538,...,-0.338030,-0.228925,-1.910580,-2.392154,NaN,NaN,NaN,NaN,NaN,NaN
RPS6KA5_S212s,0.157644,0.965682,0.026429,-0.107248,-1.706914,-1.616308,-0.205057,2.482531,0.105787,-1.490049,...,0.760187,-2.253563,-2.401767,-1.397919,0.168627,-0.127457,-1.482959,NaN,NaN,NaN
EIF2AK4_S572s,-0.162349,-0.836640,-1.160616,NaN,NaN,NaN,-0.143177,0.811225,-0.591035,NaN,...,-0.801032,NaN,NaN,NaN,-0.631002,-0.632279,-1.815198,NaN,NaN,NaN


In [43]:
df2 = df62.T
df2.index.names = ['Sample ID']
#df2 = df2.dropna(how = "all", axis = 1)
#df2.columns = df2.columns.str.replace(r' ', "_", regex = True)
df2.head()

Gene_Psite,FAM184B_S976s,TRIM28_S489s,ARHGEF35_S450s,RPS6KA5_S212s,EIF2AK4_S572s,SRRT_S136s,BCL9_S441s,TP53BP1_S899s,FAM21A_S836s,MAPT_S726s T731t,...,FAM83H_S1024s,WIZ_S289s S294s,DTNBP1_S11s,C1ORF174_S28s,RBM5_S18s,INTS1_S13s,RBM10_S596s,KCTD1_S617s S620s,TAF3_S199s,PRR12_S386s
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,-0.380722,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,2.619281,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,0.015909,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432
E2-A10A,-1.798008,1.724957,0.282548,-0.107248,NaN,-0.957562,-0.720724,2.945330,1.488119,-0.200996,...,0.025973,-4.562762,-0.074354,-3.620344,-3.924615,-2.360498,1.057205,-0.342441,0.022684,-1.283214
BH-A18Q,-3.448592,-0.394081,-1.379211,-1.706914,NaN,-1.496537,-1.413600,-0.731897,-1.033303,-4.629940,...,-0.723806,-2.083165,2.480397,-0.938229,-1.336731,-1.150629,-1.237611,-0.764263,-1.348868,-1.138491


# Get mutation data

In [44]:
mutations = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable01.csv")
mutations = mutations.set_index("Sample ID")
mutations = mutations[["PIK3CA Mutation"]]
mutations = mutations.rename(columns = {"PIK3CA Mutation" : "PIK3CA_Mutation"})
mutations =  mutations.replace(np.nan, 'Wildtype', regex=True)
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense_Mutation|Somatic|p.H1047R
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense_Mutation|Somatic|p.E545K
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [45]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E542K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E545K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('H1047R')] = "Hotspot"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Hotspot
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Hotspot
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [46]:
merged = pd.merge(df2, mutations, on = "Sample ID", how = "outer")
merged.head()

,FAM184B_S976s,TRIM28_S489s,ARHGEF35_S450s,RPS6KA5_S212s,EIF2AK4_S572s,SRRT_S136s,BCL9_S441s,TP53BP1_S899s,FAM21A_S836s,MAPT_S726s T731t,...,WIZ_S289s S294s,DTNBP1_S11s,C1ORF174_S28s,RBM5_S18s,INTS1_S13s,RBM10_S596s,KCTD1_S617s S620s,TAF3_S199s,PRR12_S386s,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948,Wildtype
AO-A12D,-4.861437,-2.901710,-0.330563,-0.322597,-0.077631,-0.653201,1.087451,-0.894184,-0.384336,-0.999738,...,2.320247,0.358528,0.155386,-0.322597,NaN,1.055586,-0.714941,-2.268383,-0.834436,Wildtype
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694,Missense_Mutation|Somatic|p.R108H
C8-A131,NaN,0.710636,1.587940,0.295329,NaN,0.172457,0.956379,0.506669,NaN,-7.647102,...,1.007986,0.801561,NaN,NaN,0.450148,2.160523,NaN,-1.198792,0.270755,Missense_Mutation|Somatic|p.R108H
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432,Wildtype


In [47]:
wt_hotspot = merged.loc[(merged['PIK3CA_Mutation'] == "Wildtype") |
                (merged['PIK3CA_Mutation'] == "Hotspot")]

In [48]:
test_cols = list(wt_hotspot.columns[:-1])
hotspotdf = u.wrap_ttest(wt_hotspot, "PIK3CA_Mutation", comparison_columns = test_cols , correction_method="fdr_bh", return_all=True, pval_return_corrected=True)

In [53]:

hotspotdf.tail(20)

,Comparison,P_Value
42,HEATR3_T340t,0.070013
43,METTL3_S43s,0.078919
44,ANXA11_S342s,0.086473
45,PHF20L1_T420t,0.087137
46,MPG_C61c T67t,0.087137
47,FAM21A_S836s,0.087137
48,ARHGEF35_S450s,0.087137
49,PRR12_S386s,0.087137
50,RXRA_T23t,0.102104
51,RAVER1_S584s,0.104228


# All Missense

In [14]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('Missense')] = "Missense"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('Hotspot')] = "Missense"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [15]:
merged = pd.merge(df2, mutations, on = "Sample ID", how = "outer")
merged.head()

,FAM184B_S976s,TRIM28_S489s,ARHGEF35_S450s,RPS6KA5_S212s,EIF2AK4_S572s,SRRT_S136s,BCL9_S441s,TP53BP1_S899s,FAM21A_S836s,MAPT_S726s T731t,...,WIZ_S289s S294s,DTNBP1_S11s,C1ORF174_S28s,RBM5_S18s,INTS1_S13s,RBM10_S596s,KCTD1_S617s S620s,TAF3_S199s,PRR12_S386s,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948,Wildtype
AO-A12D,-4.861437,-2.901710,-0.330563,-0.322597,-0.077631,-0.653201,1.087451,-0.894184,-0.384336,-0.999738,...,2.320247,0.358528,0.155386,-0.322597,NaN,1.055586,-0.714941,-2.268383,-0.834436,Wildtype
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694,Missense
C8-A131,NaN,0.710636,1.587940,0.295329,NaN,0.172457,0.956379,0.506669,NaN,-7.647102,...,1.007986,0.801561,NaN,NaN,0.450148,2.160523,NaN,-1.198792,0.270755,Missense
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432,Wildtype


In [16]:
wt_missense = merged.loc[(merged['PIK3CA_Mutation'] == "Wildtype") |
                (merged['PIK3CA_Mutation'] == "Missense")]

In [29]:
test_cols = list(wt_missense.columns[:-1])
missensedf = u.wrap_ttest(wt_missense, "PIK3CA_Mutation", comparison_columns = test_cols , correction_method="fdr_bh", return_all=False, pval_return_corrected=True)
missensedf.to_csv("Brca_mertins_pvals.csv",index = False)
missensedf

,Comparison,P_Value
0,HUWE1_S3919s T3924t T3927t,0.000173
1,FAM184B_S976s,0.000187
2,BCL9_S441s,0.000187
3,UBXN2B_S66s,0.000187
4,RPS6KA5_S212s,0.000187
...,...,...
57,KHDRBS1_S20s,0.012683
58,FAM83H_S1024s,0.015346
59,C1ORF174_S28s,0.015346
60,UBXN7_S106s,0.029225


# Double check

In [18]:
#Read in Supplemental Table 18. Has phosphosites for PIK3CA that are significant. 
sup18 = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable18.csv")
sup18["phospho_site"] =  sup18["RefSeq"] + "_" + sup18["Phosphosite"]
sup18 = sup18[["Gene Name","FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)", "phospho_site","RefSeq","RefSeq_psite_#sites_#localized_1staa_lastaa"]]
sup18 = sup18.rename(columns = {"FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)" : "Significant", "Gene Name" : "Gene_Name", "RefSeq_psite_#sites_#localized_1staa_lastaa" : "Full_ID"})

sup18

,Gene_Name,Significant,phospho_site,RefSeq,Full_ID
0,FAM184B,True,NP_056503_S976s,NP_056503,NP_056503_S976s _1_1_976_976
1,TRIM28,True,NP_005753_S489s,NP_005753,NP_005753_S489s _1_1_489_489
2,ARHGEF35,True,NP_001003702_S450s,NP_001003702,NP_001003702_S450s _1_1_450_450
3,RPS6KA5,True,NP_004746_S212s,NP_004746,NP_004746_S212s _1_1_212_212
4,EIF2AK4,True,NP_001013725_S572s,NP_001013725,NP_001013725_S572s _1_1_572_572
...,...,...,...,...,...
25856,FAM171A1,False,NP_001010924_S525s,NP_001010924,NP_001010924_S525s _1_0_524_525
25857,TRIM2,False,NP_001123539_S424s S428s,NP_001123539,NP_001123539_S424sS428s_2_2_424_428
25858,MAP2,False,NP_002365_T1649t S1653s,NP_002365,NP_002365_T1649tS1653s_2_2_1649_1653
25859,MAP2,False,NP_002365_S1155s,NP_002365,NP_002365_S1155s _1_0_1154_1155


In [19]:
sup18_T = sup18.loc[sup18["Significant"] == True]
sup18_T = sup18_T.set_index("Full_ID")
sup18_T

,Gene_Name,Significant,phospho_site,RefSeq
Full_ID,,,,
NP_056503_S976s _1_1_976_976,FAM184B,True,NP_056503_S976s,NP_056503
NP_005753_S489s _1_1_489_489,TRIM28,True,NP_005753_S489s,NP_005753
NP_001003702_S450s _1_1_450_450,ARHGEF35,True,NP_001003702_S450s,NP_001003702
NP_004746_S212s _1_1_212_212,RPS6KA5,True,NP_004746_S212s,NP_004746
NP_001013725_S572s _1_1_572_572,EIF2AK4,True,NP_001013725_S572s,NP_001013725
...,...,...,...,...
NP_001073922_S13s _1_1_13_13,INTS1,True,NP_001073922_S13s,NP_001073922
NP_001191397_S596s _1_1_596_596,RBM10,True,NP_001191397_S596s,NP_001191397
NP_001136202_S617sS620s_2_2_617_620,KCTD1,True,NP_001136202_S617s S620s,NP_001136202


In [20]:
sup4 = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable04.csv")
cols = [1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,27]
sup4 = sup4.drop(sup4.columns[cols],axis=1)
sup4 = sup4.rename(columns = {"accessionNumber_VMsites_numVMsitesPresent_numVMsitesLocalizedBest_earliestVMsiteAA_latestVMsiteAA" : "Full_ID"})
sup4 = sup4.set_index("Full_ID")
sup4

,variableSites,accession_number,geneName,AO-A12D.01TCGA,C8-A131.01TCGA,AO-A12B.01TCGA,E2-A10A.02TCGA,BH-A18Q.02TCGA,C8-A130.02TCGA,C8-A138.03TCGA,...,AO-A12B.34TCGA,A2-A0SW.35TCGA,AO-A0JL.35TCGA,BH-A0BV.35TCGA,A2-A0YM.36TCGA,BH-A0C7.36TCGA,A2-A0SX.36TCGA,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
Full_ID,,,,,,,,,,,,,,,,,,,,,
NP_001611_S1007s _1_1_1007_1007,S1007s,NP_001611,AHNAK,-0.919089,-2.565861,0.177221,0.203602,-0.086607,-2.700983,1.301494,...,0.805357,4.389396,1.338505,3.450978,0.200324,1.728556,1.092898,-1.153228,0.234135,2.595100
NP_001611_S1010s _1_1_1010_1010,S1010s,NP_001611,AHNAK,-1.271514,-1.726458,-0.320742,-0.523359,-0.315189,-2.208631,1.820337,...,-0.124880,0.070029,0.495032,0.776968,1.017243,2.213385,0.444430,-0.331621,0.347756,2.527968
NP_001611_S1023s _1_1_1023_1023,S1023s,NP_001611,AHNAK,NaN,NaN,NaN,NaN,NaN,NaN,-0.302638,...,0.880172,0.812259,0.364182,1.626879,1.985442,-0.342381,1.767384,NaN,NaN,NaN
NP_001611_S1042s _1_1_1042_1042,S1042s,NP_001611,AHNAK,-2.298519,-0.370305,-1.169382,0.330244,0.819633,-0.419166,1.570435,...,-0.795385,0.797827,1.628386,1.382329,NaN,NaN,NaN,NaN,NaN,NaN
NP_001611_S1068s _1_1_1068_1068,S1068s,NP_001611,AHNAK,-0.802335,-2.023904,-1.223738,-0.465794,-1.585542,-1.411369,1.394314,...,-0.915370,2.065803,3.490470,2.797509,0.485597,2.001793,-0.352144,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_543009_T210t _1_0_208_210,T210t,NP_543009,GPR78,0.014945,2.483162,-0.324249,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NP_001017372_S217sT218tT224t_3_0_217_228,S217s T218t T224t,NP_001017372,SLC27A6,-0.698553,-1.693688,-0.175210,0.729908,0.613301,-0.054275,NaN,...,0.504689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NP_954857_T47t _1_0_45_47,T47t,NP_954857,C19ORF48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df62 = pd.merge(sup18_T, sup4, on = "Full_ID", how = "inner")
df62.columns = df62.columns.str.replace(r'...TCGA', "", regex = True)
cols = ["Significant","RefSeq","accession_number","geneName","Gene_Name","variableSites","phospho_site"]
df62 = df62.drop(cols, axis = 1) 
df62

,AO-A12D,C8-A131,AO-A12B,E2-A10A,BH-A18Q,C8-A130,C8-A138,E2-A154,BH-A0BZ,A8-A09I,...,AO-A12B,A2-A0SW,AO-A0JL,BH-A0BV,A2-A0YM,BH-A0C7,A2-A0SX,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
Full_ID,,,,,,,,,,,,,,,,,,,,,
NP_056503_S976s _1_1_976_976,-4.916841,-2.966657,-2.954333,-1.798008,-3.448592,2.449975,-4.629509,0.906083,-3.190797,-3.790538,...,-2.441299,NaN,NaN,NaN,-3.006837,-2.146746,-4.983482,3.717143,-1.413374,-2.761566
NP_005753_S489s _1_1_489_489,-2.624999,0.519513,-1.597210,1.724957,-0.394081,-2.106161,1.506175,2.703867,-0.561201,-1.867217,...,-1.151106,-0.828894,-2.297088,0.855144,-0.939327,0.467334,-0.956579,3.014077,2.552447,1.402929
NP_001003702_S450s _1_1_450_450,-0.017487,0.365749,-0.562710,0.282548,-1.379211,-1.236422,-1.994830,0.811225,-2.304321,-2.450538,...,-0.338030,-0.228925,-1.910580,-2.392154,NaN,NaN,NaN,NaN,NaN,NaN
NP_004746_S212s _1_1_212_212,0.157644,0.965682,0.026429,-0.107248,-1.706914,-1.616308,-0.205057,2.482531,0.105787,-1.490049,...,0.760187,-2.253563,-2.401767,-1.397919,0.168627,-0.127457,-1.482959,NaN,NaN,NaN
NP_001013725_S572s _1_1_572_572,-0.162349,-0.836640,-1.160616,NaN,NaN,NaN,-0.143177,0.811225,-0.591035,NaN,...,-0.801032,NaN,NaN,NaN,-0.631002,-0.632279,-1.815198,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_001073922_S13s _1_1_13_13,NaN,NaN,NaN,-2.360498,-1.150629,-0.239220,2.129739,1.240344,-3.783202,-2.712916,...,NaN,-1.142280,-1.226138,-0.960937,NaN,NaN,NaN,NaN,NaN,NaN
NP_001191397_S596s _1_1_596_596,NaN,NaN,NaN,1.057205,-1.237611,-0.786556,NaN,NaN,NaN,0.051419,...,-1.352963,NaN,NaN,NaN,-0.547437,0.345710,-1.152721,-1.333082,-1.232017,-1.546246
NP_001136202_S617sS620s_2_2_617_620,-0.780714,0.385914,0.263137,-0.342441,-0.764263,1.840157,-0.914302,-0.907511,-1.147213,-0.651378,...,0.386115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df2 = df62.T
df2.index.names = ['Sample ID']
#df2 = df2.dropna(how = "all", axis = 1)
#df2.columns = df2.columns.str.replace(r' ', "_", regex = True)
df2.head()

Full_ID,NP_056503_S976s _1_1_976_976,NP_005753_S489s _1_1_489_489,NP_001003702_S450s _1_1_450_450,NP_004746_S212s _1_1_212_212,NP_001013725_S572s _1_1_572_572,NP_056992_S136s _1_1_136_136,XP_003960280_S441s _1_1_441_441,NP_001135452_S899s _1_1_899_899,NP_001005751_S836s _1_0_836_837,NP_058519_S726sT731t_2_1_726_733,...,NP_940890_S1024s _1_1_1024_1024,NP_067064_S289sS294s_2_2_289_294,NP_115498_S11s _1_1_11_11,NP_997239_S28s _1_1_28_28,NP_005769_S18s _1_1_18_18,NP_001073922_S13s _1_1_13_13,NP_001191397_S596s _1_1_596_596,NP_001136202_S617sS620s_2_2_617_620,NP_114129_S199s _1_1_199_199,NP_065770_S386s _1_1_386_386
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,-0.380722,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,2.619281,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,0.015909,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432
E2-A10A,-1.798008,1.724957,0.282548,-0.107248,NaN,-0.957562,-0.720724,2.945330,1.488119,-0.200996,...,0.025973,-4.562762,-0.074354,-3.620344,-3.924615,-2.360498,1.057205,-0.342441,0.022684,-1.283214
BH-A18Q,-3.448592,-0.394081,-1.379211,-1.706914,NaN,-1.496537,-1.413600,-0.731897,-1.033303,-4.629940,...,-0.723806,-2.083165,2.480397,-0.938229,-1.336731,-1.150629,-1.237611,-0.764263,-1.348868,-1.138491


In [23]:
mutations = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable01.csv")
mutations = mutations.set_index("Sample ID")
mutations = mutations[["PIK3CA Mutation"]]
mutations = mutations.rename(columns = {"PIK3CA Mutation" : "PIK3CA_Mutation"})
mutations =  mutations.replace(np.nan, 'Wildtype', regex=True)

mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense_Mutation|Somatic|p.H1047R
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense_Mutation|Somatic|p.E545K
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [24]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E542K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E545K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('H1047R')] = "Hotspot"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Hotspot
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Hotspot
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [25]:
merged = pd.merge(df2, mutations, on = "Sample ID", how = "outer")
merged.head()

,NP_056503_S976s _1_1_976_976,NP_005753_S489s _1_1_489_489,NP_001003702_S450s _1_1_450_450,NP_004746_S212s _1_1_212_212,NP_001013725_S572s _1_1_572_572,NP_056992_S136s _1_1_136_136,XP_003960280_S441s _1_1_441_441,NP_001135452_S899s _1_1_899_899,NP_001005751_S836s _1_0_836_837,NP_058519_S726sT731t_2_1_726_733,...,NP_067064_S289sS294s_2_2_289_294,NP_115498_S11s _1_1_11_11,NP_997239_S28s _1_1_28_28,NP_005769_S18s _1_1_18_18,NP_001073922_S13s _1_1_13_13,NP_001191397_S596s _1_1_596_596,NP_001136202_S617sS620s_2_2_617_620,NP_114129_S199s _1_1_199_199,NP_065770_S386s _1_1_386_386,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948,Wildtype
AO-A12D,-4.861437,-2.901710,-0.330563,-0.322597,-0.077631,-0.653201,1.087451,-0.894184,-0.384336,-0.999738,...,2.320247,0.358528,0.155386,-0.322597,NaN,1.055586,-0.714941,-2.268383,-0.834436,Wildtype
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694,Missense_Mutation|Somatic|p.R108H
C8-A131,NaN,0.710636,1.587940,0.295329,NaN,0.172457,0.956379,0.506669,NaN,-7.647102,...,1.007986,0.801561,NaN,NaN,0.450148,2.160523,NaN,-1.198792,0.270755,Missense_Mutation|Somatic|p.R108H
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432,Wildtype


In [26]:
wt_hotspot = merged.loc[(merged['PIK3CA_Mutation'] == "Wildtype") |
                (merged['PIK3CA_Mutation'] == "Hotspot")]

In [27]:
test_cols = list(wt_hotspot.columns[:-1])
hotspotdf = u.wrap_ttest(wt_hotspot, "PIK3CA_Mutation", comparison_columns = test_cols , correction_method="fdr_bh", return_all=True, pval_return_corrected=False)
hotspotdf.tail()

,Comparison,P_Value
57,NP_001229827_S796s _1_1_796_796,0.121627
58,NP_113584_S1368s _1_1_1368_1368,0.151835
59,NP_002264_S475s _1_1_475_475,0.187491
60,NP_940890_S1024s _1_1_1024_1024,0.401119
61,NP_056377_S106s _1_1_106_106,0.516800


# All missense check

In [20]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('Missense')] = "Missense"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('Hotspot')] = "Missense"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [21]:
merged = pd.merge(df2, mutations, on = "Sample ID", how = "outer")
merged.head()

,FAM184B_S976s,TRIM28_S489s,ARHGEF35_S450s,RPS6KA5_S212s,EIF2AK4_S572s,SRRT_S136s,BCL9_S441s,TP53BP1_S899s,FAM21A_S836s,MAPT_S726s T731t,...,WIZ_S289s S294s,DTNBP1_S11s,C1ORF174_S28s,RBM5_S18s,INTS1_S13s,RBM10_S596s,KCTD1_S617s S620s,TAF3_S199s,PRR12_S386s,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948,Wildtype
AO-A12D,-4.861437,-2.901710,-0.330563,-0.322597,-0.077631,-0.653201,1.087451,-0.894184,-0.384336,-0.999738,...,2.320247,0.358528,0.155386,-0.322597,NaN,1.055586,-0.714941,-2.268383,-0.834436,Wildtype
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694,Missense
C8-A131,NaN,0.710636,1.587940,0.295329,NaN,0.172457,0.956379,0.506669,NaN,-7.647102,...,1.007986,0.801561,NaN,NaN,0.450148,2.160523,NaN,-1.198792,0.270755,Missense
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432,Wildtype


In [22]:
wt_missense = merged.loc[(merged['PIK3CA_Mutation'] == "Wildtype") |
                (merged['PIK3CA_Mutation'] == "Missense")]
wt_missense

,FAM184B_S976s,TRIM28_S489s,ARHGEF35_S450s,RPS6KA5_S212s,EIF2AK4_S572s,SRRT_S136s,BCL9_S441s,TP53BP1_S899s,FAM21A_S836s,MAPT_S726s T731t,...,WIZ_S289s S294s,DTNBP1_S11s,C1ORF174_S28s,RBM5_S18s,INTS1_S13s,RBM10_S596s,KCTD1_S617s S620s,TAF3_S199s,PRR12_S386s,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948,Wildtype
AO-A12D,-4.861437,-2.901710,-0.330563,-0.322597,-0.077631,-0.653201,1.087451,-0.894184,-0.384336,-0.999738,...,2.320247,0.358528,0.155386,-0.322597,NaN,1.055586,-0.714941,-2.268383,-0.834436,Wildtype
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694,Missense
C8-A131,NaN,0.710636,1.587940,0.295329,NaN,0.172457,0.956379,0.506669,NaN,-7.647102,...,1.007986,0.801561,NaN,NaN,0.450148,2.160523,NaN,-1.198792,0.270755,Missense
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AO-A0JL,NaN,-2.297088,-1.910580,-2.401767,NaN,-0.388704,0.122615,0.062223,-0.159214,NaN,...,NaN,1.278113,NaN,NaN,-1.226138,NaN,NaN,1.427080,NaN,Wildtype
BH-A0BV,NaN,0.855144,-2.392154,-1.397919,NaN,-0.327514,-2.291928,0.097441,0.067374,NaN,...,NaN,1.841361,NaN,NaN,-0.960937,NaN,NaN,0.957374,NaN,Missense
A2-A0YM,-3.006837,-0.939327,NaN,0.168627,-0.631002,0.128286,NaN,-0.978228,0.322790,-2.397390,...,NaN,0.596537,NaN,NaN,NaN,-0.547437,NaN,-1.468091,NaN,Wildtype


In [25]:
test_cols = list(wt_missense.columns[:-1])
missensedf_full_id = u.wrap_ttest(wt_missense, "PIK3CA_Mutation", comparison_columns = test_cols , correction_method="fdr_bh", return_all=False, pval_return_corrected=True)
missensedf_full_id.tail(32)

,Comparison,P_Value
30,WWC3_S957s,0.001754
31,RXRA_T23t,0.001857
32,PPP6R2_S796s,0.001863
33,METTL3_S43s,0.002848
34,FIP1L1_T221t,0.003051
35,RBM5_S18s,0.003124
36,NOSIP_S36s,0.003245
37,WIZ_S289s S294s,0.004466
38,PHF20L1_T420t,0.004466
39,BCL2L11_S77s,0.004466
